This notebook is developed as a user interface to the rt2_assignment1_1 ROS package.

It is composed of seven main components:

      1- Initializations
      
      2- Implementing the robot's starting/stopping using two buttons

      3- Implementing the control of the robot's linear and angular velocity using sliders

      4- Implementing the control of the robot using (forward, turn right, backward, turn left, stop) buttons

      5- Visualizing cmd_vel vs. actual velocity

      6- Visualizing the robot's position
      
      7- Bar plot displaying reached and cancelled targets

      8- Histogram plot displaying time required to reach targets

#### Initializations

First, launch the ros simulation:

$ roslaunch rt2_assignment1_1 sim.launch

Import all required libraries and ros messages

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from matplotlib import animation, rc
import ipywidgets as widgets
from ipywidgets import Button, ButtonStyle, Layout, VBox, HBox, GridBox
import time

In [2]:
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
from rt2_assignment1_1.srv import Command
from rt2_assignment1_1.msg import GoToPointActionResult


Initializing the ros node, the client to the user_interface service, and the cmd_vel publisher

In [3]:
rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

Throughout the testing of this user interface notebook, we need to store all the values related to reached and cancelled targets as well as time durations to reach targets. These data will be plotted later in the end in sections 6 and 7. 

In [4]:
num_reached=0  ##number of reached targets
num_unreached=0  ##number of cancelled targets
durations=[]   ##list to store time duration to reach every target

##################################################
#the callback function for the /go_to_point_ac/result subscriber
def goal_result_callback(msg):
    global num_reached
    global num_unreached
    
    if msg.result.reached == True:
        #if the goal was reached, increase the number of reached targets
        num_reached = num_reached+1
        #calculate the time duration in sec to get to the target and append it to the list
        start_time = (msg.status.goal_id.stamp.secs*1000000000) + msg.status.goal_id.stamp.nsecs
        end_time = (msg.header.stamp.secs*1000000000) + msg.header.stamp.nsecs
        duration_sec = (end_time - start_time)/1000000000
        durations.append(duration_sec)
    else:
        #if the goal is cancelled, increase the number of cancelled targets
        num_unreached = num_unreached+1
    

#Define the subscriber to /go_to_point_ac/result to observe the goals results
jr.subscribe('/go_to_point_ac/result', GoToPointActionResult, goal_result_callback)

#### Implementing the robot's starting/stopping using two buttons

defining the start and stop buttons

In [5]:
b_start = Button(description='Start', layout=Layout(width='auto', align='center', grid_area='b_start'), style=ButtonStyle(button_color='green'))
b_stop = Button(description='Stop', layout=Layout(width='auto', align='center', grid_area= 'b_stop'), style=ButtonStyle(button_color='red'))

In [6]:
display(b_start)
display(b_stop)

Button(description='Start', layout=Layout(grid_area='b_start', width='auto'), style=ButtonStyle(button_color='…

Button(description='Stop', layout=Layout(grid_area='b_stop', width='auto'), style=ButtonStyle(button_color='re…

defining the on_click functions for the two buttons

In [7]:
def on_start_click(b_start):
    #call the service with "start" command
    ui_client("start")
    
def on_stop_click(b_stop):
    #call the service with "stop" command
    ui_client("stop")
    
b_start.on_click(on_start_click)
b_stop.on_click(on_stop_click)

#### Implementing the control of the robot's linear and angular velocity using sliders

In [8]:
msg = Twist()

##Defining the two sliders for linear and angular velocities
a = widgets.FloatSlider(min=-1.0, max=1.0)
display(a)

b = widgets.FloatSlider(min=-1.0, max=1.0)
display(b)

## Defining the callback functions for change in each slider
def on_value_change(change):
    #stop the robot
    ui_client("stop")
    #publish the new linear velocity value 
    global msg
    msg.linear.x = change['new']
    vel_pub.publish(msg)
    
def on_valueang_change(change):
    #stop the robot
    ui_client("stop")
    #publish the new angular velocity value
    global msg
    msg.angular.z = change['new']
    vel_pub.publish(msg)

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')

FloatSlider(value=0.0, max=1.0, min=-1.0)

FloatSlider(value=0.0, max=1.0, min=-1.0)

#### Implementing the control of the robot using (forward, turn right, backward, turn left, stop) buttons

Defining the five buttons with appropriate icon for each button

In [9]:
b_up = Button(description='', 
              layout=Layout(width='auto', grid_area='b_up'), 
              style=ButtonStyle(button_color='lightgreen'),
             icon = 'angle-double-up')
b_right = Button(description='', 
                 layout=Layout(width='auto', grid_area= 'b_right'), 
                 style=ButtonStyle(button_color='lightgreen'),
                icon = 'angle-double-right')
b_left = Button(description='', 
                layout=Layout(width='auto', grid_area='b_left'), 
                style=ButtonStyle(button_color='lightgreen'),
               icon = 'angle-double-left')
b_back = Button(description='', 
                layout=Layout(width='auto', grid_area= 'b_back'), 
                style=ButtonStyle(button_color='lightgreen'),
               icon = 'angle-double-down')
b_stopp = Button(description='Stop', 
                 layout=Layout(width='auto', grid_area='b_stopp'), 
                 style=ButtonStyle(button_color='orange'))

Placing the buttons in appropriate places using GridBox

In [10]:
GridBox(children=[b_up, b_right, b_left, b_back, b_stopp], 
        layout = Layout(width = '50%',
                        grid_template_rows = 'auto auto auto',
                        grid_template_columns = '25% 25% 25%',
                        grid_template_areas = '''
                        ". b_up ."
                        "b_left b_stopp b_right"
                        ". b_back ."
                        '''
                        )
       )

GridBox(children=(Button(icon='angle-double-up', layout=Layout(grid_area='b_up', width='auto'), style=ButtonSt…

Defining the callback function for each button

In [11]:
def on_forward_click(b_up):
    #stop the robot
    ui_client("stop")
    # publish forward linear velocity
    global msg
    msg.linear.x = 0.4
    msg.angular.z = 0.0
    vel_pub.publish(msg)
    
def on_right_click(b_right):
    #stop the robot
    ui_client("stop")
    #publish positive angular velocity with small forward velocity
    global msg
    msg.linear.x = 0.1
    msg.angular.z = 0.4
    vel_pub.publish(msg)
    
def on_left_click(b_left):
    #stop the robot
    ui_client("stop")
    #publish negative angular velocity with small forward velocity
    global msg
    msg.linear.x = 0.1
    msg.angular.z = -0.4
    vel_pub.publish(msg)
    
def on_backward_click(b_back):
    #stop the robot
    ui_client("stop")
    # publish backward linear velocity
    global msg
    msg.linear.x = -0.4
    msg.angular.z = 0.0
    vel_pub.publish(msg)
    
def on_stopp_click(b_stopp):
    #stop the robot
    ui_client("stop")
    #publish zero velocity (This is done to cancel any effect the velocity slider may still doing)
    global msg
    msg.linear.x = 0.0
    msg.angular.z = 0.0
    vel_pub.publish(msg)
    
b_up.on_click(on_forward_click)
b_right.on_click(on_right_click)
b_left.on_click(on_left_click)
b_back.on_click(on_backward_click)
b_stopp.on_click(on_stopp_click)

#### Visualizing cmd_vel vs. actual velocity

Here I plot the cmd_vel vs actual robot velocity in real time. Velocities are updated in the plot only each 1 second in order to avoid filling up the memory with a lot of received velocity values from the subscribers. 

Actual robot's velocity is retrieved from /odom topic in the field twist

cmd_vel value is retrieved from /cmd_vel topic

In the angular velocity plot, you may notice that the actual and cmd_vel values are opposite in sin. The reason for this can be that the sign convention of angular.z in /odom is different from /cmd_vel (I am not sure)

In [12]:
# First set up the figure and the axis for plotting the linear velocities
fig, ax = plt.subplots()

ax.set_xlim(( -2, 500)) #plot the velocities for the first 500 seconds 
ax.set_ylim((-1, 1))  #set y limits between -1 and 1
ax.set_title('Linear cmd_vel vs actual velocity')
ax.set_ylabel('Linear velocity')
ax.set_xlabel('time (S)')

line1, = ax.plot([],[], 'r--', label='actual velocity')
line2, = ax.plot([],[], 'b--', label='cmd_vel')
ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Setting up the figure and the axis for plotting the angular velocities
fig2, ax2 = plt.subplots()

ax2.set_xlim(( -2, 500)) #plot the velocities for the first 500 seconds 
ax2.set_ylim((-1, 1))    #set y limits between -1 and 1
ax2.set_title('Angular cmd_vel vs actual velocity')
ax2.set_ylabel('angular velocity')
ax2.set_xlabel('time (S)')

line3, = ax2.plot([],[], 'r--', label='actual velocity')
line4, = ax2.plot([],[], 'b--', label='cmd_vel')
ax2.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
#inintializing the varibales used for updating the velocity values and the plots
#Also the variables used for storing the robot's x and y position are initialized and updated for plotting the position 
# in the next section

count=0            #count to update the x data of the plots
current_act_vel_lin=0   #current value of actual linear velocity
current_cmd_vel_lin=0   #current value of linear cmd_vel
current_act_vel_ang=0   #current value of actual angualr velocity
current_cmd_vel_ang=0   #current value of angualr cmd_vel
current_x_pos=0         #current value of robot's x position
current_y_pos=0         #current value of robot's y position

x_data=[]  #list to store the x data for the plots
act_vel_data=[]  #list to store the actual linear velocity data for plotting
cmd_vel_data=[]  #list to store the linear cmd_vel data for plotting
act_vel_data_ang=[]  #list to store the actual angular velocity data for plotting
cmd_vel_data_ang=[]  #list to store the angular cmd_vel data for plotting


##############################################

#the intialization functions for the four plots
def init1():
    line1.set_data([],[])
    return (line1,)

def init2():
    line2.set_data([],[])
    return (line2,)

def init3():
    line3.set_data([],[])
    return (line3,)

def init4():
    line4.set_data([],[])
    return (line4,)
##############################################


#the callback function for /odom subscriber
def actual_vel_callback(msg):
    global current_act_vel_lin
    global current_act_vel_ang
    global current_x_pos
    global current_y_pos
    #Get current values of actual valocity and position
    current_act_vel_lin = msg.twist.twist.linear.x
    current_act_vel_ang = msg.twist.twist.angular.z
    current_x_pos= msg.pose.pose.position.x
    current_y_pos=msg.pose.pose.position.y

    
#the callback function for /cmd_vel subscriber
def cmd_vel_callback(msg):
    global current_cmd_vel_lin
    global current_cmd_vel_ang
    #get current values of cmd_vel
    current_cmd_vel_lin = msg.linear.x
    current_cmd_vel_ang = msg.angular.z
    

##################################################


#the function to update the linear actual velocity plot
def animate1(i):
    global count
    ## Append the recent values of actual and cmd velocities and update the count
    ## This is implemented here to plot the velocities only at each 1 second (the interval of animate function)
    x_data.append(count)
    act_vel_data.append(current_act_vel_lin)
    cmd_vel_data.append(current_cmd_vel_lin)
    act_vel_data_ang.append(current_act_vel_ang)
    cmd_vel_data_ang.append(current_cmd_vel_ang)
    count=count+1
    
    line1.set_data(x_data, act_vel_data)
    return (line1,)

#the function to update the linear cmd_vel plot
def animate2(i):
    line2.set_data(x_data, cmd_vel_data)
    return (line2,)

#the function to update the angular actual velocity plot
def animate3(i):
    line3.set_data(x_data, act_vel_data_ang)
    return (line3,)

#the function to update the angular cmd_vel plot
def animate4(i):
    line4.set_data(x_data, cmd_vel_data_ang)
    return (line4,)

####################################################


# subscribers to /odom and /cmd_vel topics
jr.subscribe('/odom', Odometry, actual_vel_callback)
jr.subscribe('/cmd_vel', Twist, cmd_vel_callback)

In [15]:
# call the animator. blit=True means only re-draw the parts that have changed.
# The animator functions for all the four plots with a rate of 1 Hz

anim1 = animation.FuncAnimation(fig, animate1, init_func=init1,
                               frames=100, interval=1000, blit=True)
anim2 = animation.FuncAnimation(fig, animate2, init_func=init2,
                               frames=100, interval=1000, blit=True)
anim3 = animation.FuncAnimation(fig2, animate3, init_func=init3,
                               frames=100, interval=1000, blit=True)
anim4 = animation.FuncAnimation(fig2, animate4, init_func=init4,
                               frames=100, interval=1000, blit=True)

#### Visualizing the robot's position

Here I plot the robot's x and y position in real time. Position is updated in the plot only each 0.5 second in order to avoid filling up the memory with a lot of received position values from the /odom subscriber.

This section is dependent on the previous section because the /odom subscriber is defined in the previous section where the position is updated regularly

In [16]:
# First set up the figure and the axis for plotting the position
fig_pos, ax_pos = plt.subplots()

#set x and y limits to be between -5 and 5 (the limits for the random targets) 
ax_pos.set_xlim(( -5, 5)) 
ax_pos.set_ylim((-5, 5))
ax_pos.set_title('position of the robot')
ax_pos.set_ylabel('y position')
ax_pos.set_xlabel('x position')

line_pos, = ax_pos.plot([],[], 'ro')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
x_pos=[]  #list to store the x position data 
y_pos=[]  #list to store the y position data 

#the intialization function for the position plot
def init_pos():
    line_pos.set_data([],[])
    return (line_pos,)    

##################################################


#the function to update the position plot
def animate_pos(i):
    x_pos.append(current_x_pos)
    y_pos.append(current_y_pos)
    line_pos.set_data(x_pos, y_pos)
    return (line_pos,)

In [18]:
# The animator function for the position plot with a rate of 2 Hz
anim_pos = animation.FuncAnimation(fig_pos, animate_pos, init_func=init_pos,
                               frames=100, interval=500, blit=True)

#### Bar plot displaying reached and cancelled targets
#### Histogram plot displaying time required to reach targets

*Run this section in the end of testing*

Here, I display the bar plot for reached and cancelled targets as well as the histogram distribution for the time required to reach targets. 

I tried making these plots to be updated in real time using FuncAnimation(), but, I wasn't successful

In [19]:
# First set up the figure and the axis for bar plotting the number of reached and cancelled targets
fig_bar, ax_bar = plt.subplots()

ax_bar.set_title('number of reached and unreached targets')
ax_bar.set_xticks([1,0],['reached','cancelled'])

bar_tgt = ax_bar.bar(['reached','cancelled'], [num_reached,num_unreached], width=0.4, color='orange')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Set up the figure and the axis for histogram plotting the time required to reach targets
fig_hist, ax_hist = plt.subplots()

ax_hist.set_title('histogram distribution for time required to reach targets')
ax_hist.set_xlabel('duration (S)')

hist_tgt = ax_hist.hist(durations, bins=7, rwidth=0.4, align='mid')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …